# Translating sticky notes

This notebook demonstrates how to translate sticky contents using the [IBM Watson Language Translator](https://cloud.ibm.com/catalog/services/language-translator).

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-10_translate.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-01_create-a-mural.ipynb)

The sample mural for this notebook starts out looking like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-04_sentiment.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the \<workspace> and the \<id>.  And you need to join then with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [4]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [129]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

In this sample, it's assumed that all widgets in the mural are sticky notes.

In [6]:
import requests
import json

def listWidgets( mural_id, auth_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets"
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    return response_json["value"]

In [146]:
widgets_arr = listWidgets( g_mural_id, g_auth_token )
for widget in widgets_arr:
    print( widget["text"] )

I love winter!
I don't do well in the heat
Fall is colourful
Spring makes me feel optimistic
I hate the cold
Fall leaves are beautiful
Nothing is as soothing as drinking hot chocolate after being out in the cold
Swimming in the summer makes me so happy
I feel sad watching everything die off in the fall
summer is for relaxing
spring and fall are short
seasons bring a change of pace
We go south for the winter every year


## Step 4: Translate sticky-note text

### Prerequisites 

1. Create a "Lite" (free) instance of the IBM Watson Language Translator service: [IBM catalog page](https://cloud.ibm.com/catalog/services/language-translator)

2. Collect the "API key" and "URL" from the the **Manage** page for your service instance.

See: [Getting started with Language Translator](https://cloud.ibm.com/docs/language-translator?topic=language-translator-gettingstarted)

In [9]:
translator_apikey = ""
translator_url = ""

In [ ]:
!pip install ibm-watson

In [12]:
# https://cloud.ibm.com/apidocs/language-translator?code=python#authentication

from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator( translator_apikey )
language_translator = LanguageTranslatorV3(
    version="2018-05-01",
    authenticator=authenticator
)

language_translator.set_service_url( translator_url )

Translate one message just to see what the translator returns:

In [155]:
# https://cloud.ibm.com/apidocs/language-translator?code=python#translate

txt = widgets_arr[0]["text"]
translation = language_translator.translate( text = txt, model_id = "en-fr-CA" ).get_result()
print( "== English:\n" + txt + "\n\n== Translator JSON result:\n" + json.dumps( translation, indent=2, ensure_ascii=False) )

== English:
I love winter!

== Translator JSON result:
{
  "translations": [
    {
      "translation": "J'adore l'hiver!"
    }
  ],
  "word_count": 4,
  "character_count": 14
}


Now, translate the text from all the sticky notes:

In [147]:
for widget in widgets_arr:
    translation_json = language_translator.translate( text = widget["text"], model_id = "en-fr-CA" ).get_result()
    widget["text_translated"] = translation_json["translations"][0]["translation"]
    print( widget["text"] + "\n" + widget["text_translated"] + "\n" )

I love winter!
J'adore l'hiver!

I don't do well in the heat
Je ne fais pas bien dans la chaleur

Fall is colourful
L'automne est coloré

Spring makes me feel optimistic
Le printemps me rend optimiste

I hate the cold
Je déteste le froid

Fall leaves are beautiful
Les feuilles d'automne sont belles

Nothing is as soothing as drinking hot chocolate after being out in the cold
Rien n'est aussi apaisant que de boire du chocolat chaud après avoir été dans le froid

Swimming in the summer makes me so happy
Nager en été me rend si heureuse

I feel sad watching everything die off in the fall
Je suis triste de voir tout mourir à l'automne

summer is for relaxing
l'été est pour la détente

spring and fall are short
le printemps et l'automne sont courts

seasons bring a change of pace
les saisons apportent un changement de rythme

We go south for the winter every year
Nous allons au sud pour l'hiver chaque année



## Step 5: Post translated sticky notes

For each English sticky note, post a translated version.

### 5.1 Define some helper functions

In [18]:
def getTranslatedText( txt_in ):
    translation_json = language_translator.translate( text = txt_in, model_id = "en-fr-CA" ).get_result()
    return translation_json["translations"][0]["translation"]

def getTranslatedStickyJSON( widget, margin, translation_expansion ):
    translated_sticky_json = { "height" : widget["height"],
                               "width"  : widget["width"] + translation_expansion,
                               "x"      : widget["x"] + widget["width"] + margin,
                               "y"      : widget["y"] + 4*margin,
                               "shape"  : "rectangle",
                               "style"  : widget["style"],
                               "text"   : getTranslatedText( widget["text"] ) }
    return translated_sticky_json

In [68]:
def moveSticky( sticky_id, x, y, mural_id, auth_token ):
    # https://developers.mural.co/public/reference/updatestickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note/" + sticky_id
    headers = { "Content-Type" : "application/json", "Accept" : "vnd.mural.preview", "Authorization" : "Bearer " + auth_token }
    data = json.dumps( { "x" : x, "y" : y } )
    response = requests.request( "PATCH", url, headers=headers, data=data )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        
def shiftEverybodyElse( widgets_arr, widget_index, margin, translation_expansion, mural_id, auth_token ):
    base_widget = widgets_arr[ widget_index ]
    for i in range( widget_index + 1, len( widgets_arr ) ):
        widget = widgets_arr[i]
        x_org = widget["x"]
        y_org = widget["y"]
        widget["x"] += 0.5*base_widget["width"] + translation_expansion + margin
        if widget["y"] > ( base_widget["y"] + base_widget["height"] ):
            widget["y"] += translation_expansion
        moveSticky( widget["id"], widget["x"], widget["y"], mural_id, auth_token )
    return widgets_arr

In [63]:
def putSticky( sticky_json, mural_id, auth_token ):
    # https://developers.mural.co/public/reference/createstickynote
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/sticky-note"
    headers = { #"Accept"        : "application/json", 
                "Accept"        : "vnd.mural.preview",
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    response = requests.request( "POST", url, headers = headers, json = sticky_json )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ " + sticky_json["text"] + " ] " + msg )
    else:
        print( "[ " + sticky_json["text"] + " ] Success" )

In [128]:
def addTranslatedStickies( widgets_arr, mural_id, auth_token ):
    widgets_arr_ordered_L2R = widgets_arr.copy()
    widgets_arr_ordered_L2R.sort( key=lambda widget: widget["x"] )
    margin = 5
    translation_expansion = 20
    for i in range( len( widgets_arr_ordered_L2R ) ):
        widget = widgets_arr_ordered_L2R[i]
        translated_sticky_json = getTranslatedStickyJSON( widget, margin, translation_expansion )
        widgets_arr_ordered_L2R = shiftEverybodyElse( widgets_arr_ordered_L2R, i, margin, translation_expansion, mural_id, auth_token )
        putSticky( translated_sticky_json, mural_id, auth_token )

### 5.2 Post the translated sticky notes

In [149]:
import time
time.sleep(5)

# Quick!  After running this cell, switch to your browser 
# tab where the mural is to see the stickies move
# ...
addTranslatedStickies( widgets_arr, g_mural_id, g_auth_token )

[ Nous allons au sud pour l'hiver chaque année ] Success
[ Les feuilles d'automne sont belles ] Success
[ J'adore l'hiver! ] Success
[ Rien n'est aussi apaisant que de boire du chocolat chaud après avoir été dans le froid ] Success
[ L'automne est coloré ] Success
[ le printemps et l'automne sont courts ] Success
[ Je ne fais pas bien dans la chaleur ] Success
[ Le printemps me rend optimiste ] Success
[ Je suis triste de voir tout mourir à l'automne ] Success
[ Je déteste le froid ] Success
[ Nager en été me rend si heureuse ] Success
[ les saisons apportent un changement de rythme ] Success
[ l'été est pour la détente ] Success


This is what you should see happen:

<p>&nbsp;</p>

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-10_translate.gif" width="70%" title="Image of a mural" />